In [1]:
import numpy as np
from openopt import NLP
import matplotlib.pyplot as plt
%matplotlib inline

Based on papers and genetic algo.

In [7]:
def structtruss(p1, p2, xnum, ynum):
    pinjoints = []
    segments = []
    # making a dotted structure of entire space
    xspan = np.linspace(p1[0], p2[0], xnum + 1)
    yspan = np.linspace(p1[1], p2[1], ynum + 1)
    for y in yspan:
        for x in xspan:
            pinjoints.append([x, y])  # making a grid of dots
    for j in range(ynum):
        for i in range(xnum):
            inter = j * (xnum + 1)
            p1 = i + inter
            p2 = p1 + 1
            p3 = p1 + xnum + 1
            p4 = p3 + 1
            segments.extend([[p1, p2], [p1, p3], [p1, p4], [p2, p3]])
        segments.append([p2, p4])
    idx = ynum * (xnum + 1) + 1
    for j in range(xnum):
        # create segments between pinjoints.
        segments.append([idx + j - 1, idx + j])
    return np.array(pinjoints), np.array(segments)
